In [1]:
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
# nltk.download()
import torch
import torch.nn as nn
import torch.optim as optim
from model import SimilarityAware,LSD,LSDA,cos_sim,loss_function,train,train2
from input_functions import getVecDataFrame,extractData,getMaxSentLength
%load_ext autoreload
%autoreload 2

In [2]:
regexp = nltk.tokenize.RegexpTokenizer('\w+')
word_lem= nltk.stem.WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words("english")

In [3]:
Hin=100
Hout=100
the_model = LSD(Hin,Hout)
the_model.load_state_dict(torch.load("./NetModel.pth"))

the_model2 = LSDA(Hin,Hout,K_size=50,g_and_l=False)
the_model2.load_state_dict(torch.load("./NetModel2.pth"))

<All keys matched successfully>

In [4]:
the_model.eval()


LSD(
  (head_Model): SimilarityAware(
    (bi_gru): GRU(100, 100, batch_first=True, bidirectional=True)
    (att): Linear(in_features=200, out_features=1, bias=True)
    (tanh): Tanh()
    (softMax): Softmax(dim=1)
  )
  (body_Model): SimilarityAware(
    (bi_gru): GRU(100, 100, batch_first=True, bidirectional=True)
    (att): Linear(in_features=200, out_features=1, bias=True)
    (tanh): Tanh()
    (softMax): Softmax(dim=1)
  )
)

In [5]:
the_model2.eval()

LSDA(
  (LSD_model): LSD(
    (head_Model): SimilarityAware(
      (bi_gru): GRU(100, 100, batch_first=True, bidirectional=True)
      (att): Linear(in_features=200, out_features=1, bias=True)
      (tanh): Tanh()
      (softMax): Softmax(dim=1)
    )
    (body_Model): SimilarityAware(
      (bi_gru): GRU(100, 100, batch_first=True, bidirectional=True)
      (att): Linear(in_features=200, out_features=1, bias=True)
      (tanh): Tanh()
      (softMax): Softmax(dim=1)
    )
  )
  (mlpLayer): Linear(in_features=200, out_features=50, bias=True)
  (att1): myLinear(in_features=50, out_features=1, bias=False)
  (att2): myLinear(in_features=50, out_features=50, bias=False)
  (mlpLayer2): Linear(in_features=50, out_features=1, bias=True)
  (tanh): Tanh()
  (sigmoid): Sigmoid()
  (softMax1): Softmax(dim=1)
  (softMax2): Softmax(dim=0)
)

In [6]:
test_data = pd.read_json("../clickbait-17/clickbait17-train-170331/instances.jsonl", lines=True)
test_labels = pd.read_json("../clickbait-17/clickbait17-train-170331/truth.jsonl", lines=True)

test_data = test_data.sort_values(by=['id'])
test_labels = test_labels.sort_values(by=['id'])

test_data = test_data.reset_index()
test_labels = test_labels.reset_index()

display(test_data.iloc[0:5])
display(test_labels.iloc[0:5])

,index,id,postTimestamp,postText,postMedia,targetTitle,targetDescription,targetKeywords,targetParagraphs,targetCaptions
0,861,607668877594497024,Sun Jun 07 22:02:05 +0000 2015,[RT @WSJLive: This year's Tony nominees desrib...,[],Tony Nominees' Craziest Moments on Stage,"Tony Award nominees Carey Mulligan, Elisabeth ...",,"[Tony Award nominees Carey Mulligan, Elisabeth...",[]
1,1702,607671137062010880,Sun Jun 07 22:11:03 +0000 2015,[Orphaned fruit bat pups nursed back to health...,[media/607671137062010881.jpg],Going into bat to save a species: Dedication a...,North Sydney's Kukundi fruit bat shelter has e...,"Adorable,images,orphaned,bat,pups,bundled,tiny...","[Wrapped in blankets and hand-fed, these winge...",[Four fruit bat pups who were injured after be...
2,216,607672568057700352,Sun Jun 07 22:16:44 +0000 2015,[Ohio State’s Tyvis Powell learned that champi...,[media/607672568057700352.jpg],Ohio State's Tyvis Powell Thinks He's Above Cu...,In the offseason following a championship se...,"College Football, Breaking News",[In the offseason following a championship sea...,[]
3,2192,607674674168926208,Sun Jun 07 22:25:07 +0000 2015,[China’s fishermen explain why they think the ...,[],China’s fishermen explain why they think the s...,Those who sail from Hainan island say the Sout...,china sea; china fish; south china; hainan sea...,"[TANMEN, China — Little boats with noisy engin...","[Lu Yuyong, 51, looks up from a chart of the S..."
4,2285,607675444834398208,Sun Jun 07 22:28:10 +0000 2015,"[RT @BBCSport: ""I'm living my dream"" - Watch w...",[media/607675444834398208.png],BBC Sport - Lewis Hamilton 'living his dream' ...,Lewis Hamilton says Mercedes are enabling him ...,,[Lewis Hamilton says Mercedes are enabling him...,"[United""s Matteo Darmian fights for the ball w..."


,index,id,truthJudgments,truthMean,truthMedian,truthMode,truthClass
0,861,607668877594497024,"[0.0, 0.6666667, 1.0, 1.0, 0.6666667]",0.666667,0.666667,1.000000,clickbait
1,1702,607671137062010880,"[0.33333334000000003, 0.0, 0.0, 0.0, 0.0]",0.066667,0.000000,0.000000,no-clickbait
2,216,607672568057700352,"[0.0, 0.0, 0.6666667, 0.33333334000000003, 0.0]",0.200000,0.000000,0.000000,no-clickbait
3,2192,607674674168926208,"[1.0, 0.6666667, 0.33333334000000003, 0.666666...",0.600000,0.666667,0.666667,clickbait
4,2285,607675444834398208,"[0.33333334000000003, 0.6666667, 0.0, 0.0, 0.6...",0.333333,0.333333,0.000000,no-clickbait


In [7]:
columns=np.array(['id','postText','targetDescription','targetParagraphs'])
test_hd_bd=test_data[columns]
df2_test = test_hd_bd.apply(lambda x: x.astype(str).str.lower())
df2_test["id"]=pd.to_numeric(df2_test["id"])
keys=df2_test.keys()

for i in range(1,4):
    df2_test[f"{keys[i]}_proc"]=df2_test[keys[i]].apply(word_lem.lemmatize)
    df2_test[f"{keys[i]}_proc"]=df2_test[f"{keys[i]}_proc"].apply(regexp.tokenize)
    df2_test[f"{keys[i]}_proc"]=df2_test[f"{keys[i]}_proc"].apply(lambda x: [word for word in x if word not in stopwords])


In [8]:
keys1=df2_test.keys()

df2_w2vTest=df2_test[keys1[len(keys)]]
# print(keys[len(keys)-1])
print(range(len(keys)+1,len(keys1)))
for i in range(len(keys)+1,len(keys1)):
    df2_w2vTest=df2_w2vTest+df2_test[keys1[i]]

range(5, 7)


In [9]:
display(df2_w2vTest)

0       [rt, wsjlive, year, tony, nominees, desribe, c...
1       [orphaned, fruit, bat, pups, nursed, back, hea...
2       [ohio, state, tyvis, powell, learned, champion...
3       [china, fishermen, explain, think, sea, sail, ...
4       [rt, bbcsport, living, dream, watch, lewis, ha...
                              ...                        
2454    [petition, calling, kay, burley, sacking, reac...
2455    [rt, buzzfeednews, trooper, pulled, old, lady,...
2456    [rt, irin, one, ever, top, sentence, anywhere,...
2457    [video, espn, otl, study, reveals, college, at...
2458    [charts, looking, numbers, companies, behind, ...
Length: 2459, dtype: object

In [10]:
word2vecLength=100
modelW2v_test = Word2Vec(window=10, min_count=1, workers=4,vector_size=word2vecLength)
modelW2v_test.build_vocab(df2_w2vTest, progress_per=1000)
modelW2v_test.train(df2_w2vTest, total_examples=modelW2v_test.corpus_count, epochs=modelW2v_test.epochs)

(4183385, 4231170)

In [11]:
heading,body,labels_val=extractData(df2_test,test_labels)
head_len=getMaxSentLength(heading)
body_len=getMaxSentLength(body)
norm_len=max(head_len,body_len)
df_new,head_vec_test,body_vec_test=getVecDataFrame(heading,body,labels_val,norm_len,word2vecLength,modelW2v_test.wv)

In [12]:
head_vec_test.shape

torch.Size([2459, 63, 100])

In [13]:
P,Lh,Lb = the_model2.forward(head_vec_test,body_vec_test)

hbar_shape: torch.Size([2459, 50, 50])
Softmax_sum: tensor(1.0000, grad_fn=<SumBackward0>)
A_prod: torch.Size([2459, 1, 50]) Transform_comb: torch.Size([2459, 1, 50])


In [14]:
labels_val.shape

(2459,)

In [15]:
#view shape of y_pred tuple
print(P.shape,Lh.shape,Lb.shape)

torch.Size([2459, 1]) torch.Size([2459, 200]) torch.Size([2459, 200])


In [16]:
#softmax on the predictions P
y_pred = torch.nn.functional.softmax(P, dim=1)